<a href="https://colab.research.google.com/github/digitalhumanitiestextbook/dhtextbook/blob/main/chapter08/8_sentiment_analysis_practice.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
# NLTK, VADER 설치
!pip install nltk
!pip install vaderSentiment
!python -m nltk.downloader vader_lexicon

- 아래 코드를 실행한 후 나타나는 "파일 선택" 버튼을 클릭하여, 분석할 txt 파일을 업로드하세요.

In [ ]:
from google.colab import files
import os

# 파일 업로드
uploaded = files.upload()

# 업로드한 파일 경로 가져오기
file_path = list(uploaded.keys())[0]
root, ext = os.path.splitext(file_path)
output_file_path = root + '_sentiment_scores.xlsx'

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd

# NLTK의 영어 문장 토큰화 도구 다운로드
nltk.download('punkt')
nltk.download('punkt_tab')

# 파일 읽기
with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()

# 문장 단위 분리
sentences = sent_tokenize(text)

# 데이터프레임 초기화
df = pd.DataFrame({
    "number": range(1, len(sentences) + 1),
    "sentence": sentences
})

df

In [ ]:
# VADER를 활용한 감정 분석
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# SentimentIntensityAnalyzer 초기화
sid = SentimentIntensityAnalyzer()

# 감정 점수를 저장할 리스트 초기화
sentiment_v = []

# 각 문장에 대해 감정 점수 계산
for sentence in sentences:
    sentiment_v_score = sid.polarity_scores(sentence)['compound']
    sentiment_v.append(sentiment_v_score)

# 데이터프레임에 VADER 감정 점수 추가
df["sentiment_v"] = sentiment_v

# 결과 확인
df

In [ ]:
# RoBERTa 기반 모델을 활용한 감정 분석
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax

# 사용할 사전학습 감정분석 모델 이름 지정
MODEL = 'cardiffnlp/twitter-roberta-base-sentiment-latest'

# 토크나이저(문장 → 토큰)와 모델(감정분류) 불러오기
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# 하나의 문장에 대해 감정 점수를 반환하는 함수
def sentimentanalyzer(sent):
    encoded_input = tokenizer(sent, return_tensors='pt')  # 문장 토큰화 및 텐서 변환
    output = model(**encoded_input)  # 모델에 입력해서 결과 얻기
    score = output[0][0].detach().numpy()  # 결과에서 점수 추출 (Tensor → numpy)
    score = softmax(score)  # 소프트맥스 함수를 적용하여 확률로 변환

    # 점수를 각 감정(label)에 매핑
    return pd.Series(score, index=['negative', 'neutral', 'positive'])

# 여러 문장에 대해 감정 분석하는 함수
def analyze_sentiments(sentences):
    sentiment_results = {}

    # 각 문장에 대해 감정 분석 수행
    for i, sentence in enumerate(sentences):
        sentiment_results[i+1] = sentimentanalyzer(sentence)

    sentiment_indexes = {}

    for number, scores in sentiment_results.items():
        sentiment_index = scores['positive'] - scores['negative']  # positive - negative 값을 sentiment index로 정의
        # 감정 점수와 인덱스를 딕셔너리에 저장
        sentiment_indexes[number] = {
            'scores': scores,
            'sentiment_index': sentiment_index
        }

    return sentiment_indexes

# 업로드한 파일에서 추출한 문장들에 대해 RoBERTa 기반 감정 분석 수행
results = analyze_sentiments(sentences)

# RoBERTa 기반 모델의 감정 지수 결과를 데이터프레임에 추가
df["sentiment_r"] = [result["sentiment_index"] for result in results.values()]

# 결과 확인
df

In [ ]:
# VADER, RoBERTa 기반의 감정 분석 결과 평균을 sentiment_avg 컬럼에 저장
df["sentiment_avg"] = df[["sentiment_v", "sentiment_r"]].mean(axis=1)

# 결과 확인
df

- 아래 코드를 실행하면 분석 결과가 담긴 CSV 파일이 다운로드됩니다.
- 파일명은 [업로드한 파일명]_sentiment_scores.xlsx 형식이며, 실행 중인 브라우저의 다운로드 폴더에서 확인할 수 있습니다.

In [ ]:
# Google Colab 환경에서 생성된 CSV 파일을 로컬로 다운로드
from google.colab import files

df.to_excel(output_file_path, index=False)
files.download(output_file_path)

In [ ]:
# 업로드한 파일의 감정 분석 결과(sentiment_v)를 곡선으로 시각화하기 (단순이동평균의 윈도우=50)
import pandas as pd
import matplotlib.pyplot as plt

# 문장 번호와 감정 점수 컬럼 추출
sentence_numbers = df['number']  # 문장 번호
sentiment_scores_v = df['sentiment_v']  # 감정 점수(v)
moving_avg_v = sentiment_scores_v.rolling(window=50).mean()  # 윈도우 값에 따라 window 이후의 숫자 조정

plt.figure(figsize=(10, 6))  # 그래프 크기 지정
plt.plot(sentence_numbers, moving_avg_v, label='sentiment_v', linestyle='-', marker='', color='blue')  # 이동평균 곡선 그리기
plt.title(f'Sentiment Scores of {root} with Moving Average (Window=50)')  # 그래프 제목 설정
plt.xlabel('Sentence Number')  # x축 레이블 설정
plt.ylabel('Sentiment Scores')  # y축 레이블 설정
plt.legend()  # 범례 표시
plt.grid(True)  # 그리드 표시
plt.show()  # 그래프 출력

In [ ]:
# 업로드한 파일의 감정 분석 결과(sentiment_v)를 곡선으로 시각화하기
# 이동평균의 윈도우=100
moving_avg_v = sentiment_scores_v.rolling(window=100).mean() # 윈도우 값에 따라 window 이후의 숫자 조정

plt.figure(figsize=(10, 6))  # 그래프 크기 지정
plt.plot(sentence_numbers, moving_avg_v, label='sentiment_v', linestyle='-', marker='', color='blue')  # 이동평균 곡선 그리기
plt.title(f'Sentiment Scores of {root} with Moving Average (Window=100)')  # 그래프 제목 설정
plt.xlabel('Sentence Number')  # x축 레이블 설정
plt.ylabel('Sentiment Scores')  # y축 레이블 설정
plt.legend()  # 범례 표시
plt.grid(True)  # 그리드 표시
plt.show()  # 그래프 출력

In [ ]:
# 업로드한 파일의 감정 분석 결과(sentiment_v)를 곡선으로 시각화하기
# 이동평균의 윈도우=150
moving_avg_v = sentiment_scores_v.rolling(window=150).mean() # 윈도우 값에 따라 window 이후의 숫자 조정

plt.figure(figsize=(10, 6))  # 그래프 크기 지정
plt.plot(sentence_numbers, moving_avg_v, label='sentiment_v', linestyle='-', marker='', color='blue')  # 이동평균 곡선 그리기
plt.title(f'Sentiment Scores of {root} with Moving Average (Window=150)')  # 그래프 제목 설정
plt.xlabel('Sentence Number')  # x축 레이블 설정
plt.ylabel('Sentiment Scores')  # y축 레이블 설정
plt.legend()  # 범례 표시
plt.grid(True)  # 그리드 표시
plt.show()  # 그래프 출력

In [ ]:
# 업로드한 파일의 감정 분석 결과 전체를 곡선으로 시각화하기
# 이동평균의 윈도우=100
sentiments = ['sentiment_v', 'sentiment_r', 'sentiment_avg']
colors = ['blue', 'red', 'green']

plt.figure(figsize=(10, 6))  # 그래프 크기 지정

# 각 감정 지표별로 이동평균을 계산해서 곡선 그리기
for sentiment, color in zip(sentiments, colors):
    moving_avg = df[sentiment].rolling(window=100).mean()   # 이동평균(윈도우=100) 계산
    plt.plot(df['number'], moving_avg, label=sentiment, linestyle='-', color=color, alpha=0.8)  # 각 문장에 대해 곡선 그리기

plt.title(f'Sentiment Scores of {root} with Moving Average (Window=100)')  # 그래프 제목 설정
plt.xlabel('Sentence Number')  # x축 레이블 설정
plt.ylabel('Sentiment Scores')  # y축 레이블 설정
plt.legend()  # 범례 표시
plt.grid(True)  # 그리드 표시
plt.show()  # 그래프 출력